In [25]:
from Tokenizer import Tokenizer
from data_module import DataModule
from Model import Transformer_LM
import torch
import random

class Configs:
    def __init__(self):
        self.manifest_file = "total_am.txt"
        self.labels_path = "aihub_labels.csv"
        self.train_ratio = 0.8
        self.num_workers = 4
        self.batch_size = 64
        self.sample_mode = 'random' #'smart'
        self.teacher_forcing_ratio = 0.0
        
        self.num_classes = 2001
        self.d_model = 512
        self.d_ff = 2048
        self.num_heads = 4
        self.num_layers = 3
        self.model_name = "BERT"
        
bert_configs = Configs()

tokenizer = Tokenizer(label_file=bert_configs.labels_path)
tokenizer.idx2char[2000] = "<mask>"

model = Transformer_LM(
    num_classes=bert_configs.num_classes,
    d_model=bert_configs.d_model,
    d_ff=bert_configs.d_ff,
    num_heads=bert_configs.num_heads,
    num_layers=bert_configs.num_layers,
    model=bert_configs.model_name
)

In [33]:
# Test BERT Model
model.load_state_dict(torch.load("bert.pt"))

def text_corrector(sentence: str, mask_num: int):
    # sentence : 입력 문장
    # mask_num : masking 할 문자 수
    mask_indexes = random.sample(range(len(sentence)), mask_num)

    inputs = [tokenizer.sos_token] + tokenizer.encode(sentence) + [tokenizer.eos_token]
    for idx in mask_indexes:
        #inputs[idx] = 2000
        inputs[idx] = random.randrange(5, 2000)
    print("input :", tokenizer.decode(inputs[1:-1]))

    input_length = [len(inputs)]
    
    inputs = torch.Tensor(inputs).unsqueeze(0).int()

    logits, preds = model(inputs, input_length)

    for idx in mask_indexes:
        inputs[0][idx] = preds[0][idx]

    return tokenizer.decode(inputs[0][1:-1])

In [34]:
print("output :", text_corrector("이렇게 좋은 날씨에 사무실에서 뭐하는 거람", 5))

input : 이뿍게 좋은 날씨에 사낡짓홍서 뿍하는 거람
output : 이렇게 좋은 날씨에 사람 해서  하는 거람


In [39]:
gpt_configs = Configs()

gpt_configs.num_classes = 2000
gpt_configs.model_name = "GPT"

gpt_model = Transformer_LM(
    num_classes=gpt_configs.num_classes,
    d_model=gpt_configs.d_model,
    d_ff=gpt_configs.d_ff,
    num_heads=gpt_configs.num_heads,
    num_layers=gpt_configs.num_layers,
    model=gpt_configs.model_name
)

In [40]:
# Test GPT gpt_model
gpt_model.load_state_dict(torch.load("models/gpt.pt"))

def text_generator(sentence: str, length: int):
    # sentence : 입력 문장
    # length : 몇 글자를 더 이어 쓸것인지
    inputs = tokenizer.encode(sentence)
    inputs = torch.Tensor(inputs).unsqueeze(0).long()
    
    with torch.no_grad():
        for i in range(length):
            # 문장 뒤에 아무 글자나 붙여서 다음 글자까지 예측하도록 함.
            blank = torch.Tensor([[5]])
            inputs = torch.cat((inputs, blank), dim=1).long()
            input_lengths = torch.Tensor(inputs.size(1)).long()
            logits, preds = gpt_model.forward(inputs, input_lengths)
            
            # 앞의 글자들은 바꿀 것이 아니기 때문에 예측한 맨 마지막 글자만 뒤에 추가한다!
            inputs[0][-1] = preds[0][-1]
            print(tokenizer.decode(inputs))
    return tokenizer.decode(inputs)

In [42]:
text_generator("이렇게 좋을", 10)

이렇게 좋을 
이렇게 좋을 그
이렇게 좋을 그요
이렇게 좋을 그요 
이렇게 좋을 그요 요
이렇게 좋을 그요 요그
이렇게 좋을 그요 요그 
이렇게 좋을 그요 요그  
이렇게 좋을 그요 요그   
이렇게 좋을 그요 요그   요


'이렇게 좋을 그요 요그   요'